In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from kaggle_prediction_interval_birthweight.model.sampling_utils import (
    compute_highest_density_interval,
)
from kaggle_prediction_interval_birthweight.workflow.metric import score
from kaggle_prediction_interval_birthweight.data.data_processing import DataProcessor

In [3]:
data = pd.read_csv("../data/train.csv")
data_test = pd.read_csv("../data/test.csv")

In [4]:
model_type = "linear regression"

train_processor = DataProcessor(model_type=model_type)
X, y = train_processor(data)

test_processor = DataProcessor(
    model_type="neural network",
    feature_categories=train_processor.feature_categories,
    standardization_parameters=train_processor.standardization_parameters,
)
X_test = test_processor(data)

df = train_processor._enforce_feature_types(data)
df = train_processor._process_missing_data(df)
df = train_processor._subset_and_binarize(df)
X_numeric = train_processor._prepare_numerical_features(df)
X_categorical = train_processor._prepare_categorical_features(df)

KeyError: 'sds'

In [ ]:
X_numeric

In [ ]:
np.isnan(X_numeric).mean()

In [ ]:
print(X_numeric.shape)

In [ ]:
np.nanmax(X_numeric, axis=0)

In [ ]:
X_categorical

In [ ]:
X_categorical.shape

In [ ]:
np.isnan(X_categorical).any()

In [ ]:
X_categorical.max(axis=0)

In [ ]:
np.nanmax(X_categorical, axis=0)

In [ ]:
lower, upper = compute_highest_density_interval(data["DBWT"])

print(f"Coverage is {data['DBWT'].between(lower, upper).mean() * 100:.1f}%")

data["DBWT"].hist(bins=100)
plt.axvline(lower, color="orange")
plt.axvline(upper, color="orange")
plt.show()

In [ ]:
nois, coverage = score(data["DBWT"].values, lower, upper, 0.9)

print(f"NOIS is {nois:.1f}, and coverage is {coverage*100:.1f}%")